# SIADS 516: Homework 3
Version 1.0.20200303.2
### Dr. Chris Teplovs, School of Information, University of Michigan
<small><a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a>This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.

This homework assignment builds on the Spark DataFrame material we covered in class.
You will be using a compressed version of the Yelp Academic Dataset.  The data set is provided for you in the data/yelp-academic sub-folder of this notebook's directory and you should not need to download it again if you're working on the Coursera hosted notebook environment.

You might want to refer to the lecture companion notebooks (in workspace-files/resources/lecture_notebooks or equivalently via Coursera as "Ungraded Lab: Spark Core Demo" and "Ungraded Lab: Spark SQL Demo) for hints about libraries to import, how to set up a SparkSession, and how to read data files.

You will notice that there are a **lot** of reviews.  You might want to work off a small sample (i.e. use the sample() function in Spark) to work on a reduced size dataset while you're developing your solution.

**You should take care to document your work, preferably using markdown blocks. In-code commenting is also 
a good idea.**

In [1]:
"""
NAME: Paul Natland
"""

'\nNAME: Paul Natland\n'

------------------------------------------------------------------------------------------------------------------------------
#### <font color="blue">__ACKNOWLEDGEMENTS:__ I benefitted from class notebooks and lectures, searches on Stack Overflow, and conversations in SLACK</font>
------------------------------------------------------------------------------------------------------------------------------

In [36]:
#go wide screen!
from IPython.core.display import display, HTML; display(HTML("<style>.container { width:99% !important; }</style>"))

## Imports

In [2]:
from pyspark.sql import SparkSession
    #to establish the sparksession

import pyspark.sql.functions as F
    #to call certain built-in functions

from pyspark.sql.functions import col, udf, explode, rank, hour, minute, second
    #to call various fonctions on columns
    
from pyspark.sql.types import IntegerType, StringType, ArrayType
    #to clearly state value types in udf's

## Exploring data

In [3]:
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName('SIADS 516 -- HW3') \
    .getOrCreate() 

sc = spark.sparkContext

In [6]:
user = spark.read.json("data/yelp_academic/yelp_academic_dataset_user.json.gz")

user.printSchema()

root
 |-- average_stars: double (nullable = true)
 |-- compliment_cool: long (nullable = true)
 |-- compliment_cute: long (nullable = true)
 |-- compliment_funny: long (nullable = true)
 |-- compliment_hot: long (nullable = true)
 |-- compliment_list: long (nullable = true)
 |-- compliment_more: long (nullable = true)
 |-- compliment_note: long (nullable = true)
 |-- compliment_photos: long (nullable = true)
 |-- compliment_plain: long (nullable = true)
 |-- compliment_profile: long (nullable = true)
 |-- compliment_writer: long (nullable = true)
 |-- cool: long (nullable = true)
 |-- elite: string (nullable = true)
 |-- fans: long (nullable = true)
 |-- friends: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- name: string (nullable = true)
 |-- review_count: long (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)
 |-- yelping_since: string (nullable = true)



In [7]:
user.select('funny').take(10)

[Row(funny=17),
 Row(funny=22),
 Row(funny=8),
 Row(funny=4),
 Row(funny=279),
 Row(funny=2424),
 Row(funny=19356),
 Row(funny=0),
 Row(funny=693),
 Row(funny=70)]

In [8]:
business = spark.read.json("data/yelp_academic/yelp_academic_dataset_business.json.gz")

business.printSchema()

root
 |-- address: string (nullable = true)
 |-- attributes: struct (nullable = true)
 |    |-- AcceptsInsurance: string (nullable = true)
 |    |-- AgesAllowed: string (nullable = true)
 |    |-- Alcohol: string (nullable = true)
 |    |-- Ambience: string (nullable = true)
 |    |-- BYOB: string (nullable = true)
 |    |-- BYOBCorkage: string (nullable = true)
 |    |-- BestNights: string (nullable = true)
 |    |-- BikeParking: string (nullable = true)
 |    |-- BusinessAcceptsBitcoin: string (nullable = true)
 |    |-- BusinessAcceptsCreditCards: string (nullable = true)
 |    |-- BusinessParking: string (nullable = true)
 |    |-- ByAppointmentOnly: string (nullable = true)
 |    |-- Caters: string (nullable = true)
 |    |-- CoatCheck: string (nullable = true)
 |    |-- Corkage: string (nullable = true)
 |    |-- DietaryRestrictions: string (nullable = true)
 |    |-- DogsAllowed: string (nullable = true)
 |    |-- DriveThru: string (nullable = true)
 |    |-- GoodForDancing: str

In [9]:
review = spark.read.json("data/yelp_academic/yelp_academic_dataset_review.json.gz")

review.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- cool: long (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)



In [10]:
review.select('funny').take(10)

[Row(funny=1),
 Row(funny=0),
 Row(funny=0),
 Row(funny=0),
 Row(funny=0),
 Row(funny=0),
 Row(funny=4),
 Row(funny=1),
 Row(funny=0),
 Row(funny=0)]

In [11]:
review.sample(fraction=0.01).take(5)

[Row(business_id='RBGGLBIOsn8rxv5Nott4zw', cool=0, date='2014-11-12 15:29:08', funny=1, review_id='jjXMzyIn4kpBUOKsTrAp5Q', stars=5.0, text='Early this past January, I needed to find a furnished rental in the uptown Charlotte market. I really had no idea of how or where to look. I was glad when I found the Sold on Charlotte site.\nAs I recall I traded a few emails and phone calls with Chip and he was helpful in answering all my questions. We met at 626 North Graham and he showed me the unit there that was available.\nBetween Chip and Sylvia, I was "sold" on that apartment and we got the "paperwork" going. I enjoyed my stay there which lasted 6 months (3 months more that I initially signed up for).\nWhen it came time to give my notice, Sylvia gave me all the details I needed, and these were pretty much the same that were in our initial Lease Agreement. They were kind enough to give me ample notice to show the apartment during my last 30 days, without too much inconvenience. Sylvia helpe

In [12]:
review.describe().show()

+-------+--------------------+------------------+-------------------+-------------------+--------------------+------------------+------------------------------------+------------------+--------------------+
|summary|         business_id|              cool|               date|              funny|           review_id|             stars|                                text|            useful|             user_id|
+-------+--------------------+------------------+-------------------+-------------------+--------------------+------------------+------------------------------------+------------------+--------------------+
|  count|             6685900|           6685900|            6685900|            6685900|             6685900|           6685900|                             6685900|           6685900|             6685900|
|   mean|                null|0.5787708461089756|               null|0.48276671801851656|                null|3.7161993149762935|                                null|1.3541

In [13]:
checkin = spark.read.json("data/yelp_academic/yelp_academic_dataset_checkin.json.gz")

checkin.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- date: string (nullable = true)



In [14]:
checkin.take(1)

[Row(business_id='--1UhMGODdWsrMastO9DZw', date='2016-04-26 19:49:16, 2016-08-30 18:36:57, 2016-10-15 02:45:18, 2016-11-18 01:54:50, 2017-04-20 18:39:06, 2017-05-03 17:58:02')]

In [15]:
checkin.dtypes

[('business_id', 'string'), ('date', 'string')]

### <font color="magenta">Q1: How many users have received more than 5000 cool compliments?</font>

#### Answer:

In [16]:
user.filter(user['compliment_cool']>5000).count()

79

 > On the spark object I called the filter function and fed it the condition "> 5000".  This returns an RDD where just this condition is met, then 'count()' is called to, effectively, count the rows.

### <font color="magenta">Q2 (EXCLUDED): What are the names of the top 10 most complimented businesses?</font>

 > This question was excluded from the HW set (stated on SLACK)

### <font color="magenta">Q3: What are the top 10 most useful positive reviews?</font>

In [17]:
#the number of reviews that give over 4 stars
review.filter(review['stars']> 4).count()

2933082

#### Answer:

In [20]:
(review \
         #filter out all reviews with fewer than 5 stars    
         .filter(review['stars']> 4)             
        
         #sort the reviews by the 'useful' count, greatest to least
         .orderBy('useful', ascending=False)     
 
         #display the top 10 results (the most useful 5-star reviews)
        .take(10)                               
)

[Row(business_id='t-o_Sraneime4DDhWrQRBA', cool=347, date='2018-04-26 22:58:34', funny=156, review_id='1lGXlyq4MALOMx17vpBcoQ', stars=5.0, text='The Wynn Hotel. One word. Opulent. \n\nWynn, is a luxury resort and casino located on the Las Vegas Strip. Named after the casino developer Steve Wynn, this $2.7 billion resort together with Encore is the world\'s seventh-largest hotels and one of the finest in the world. \n\nHolding the title for the highest rated resort company in the world with 16 Forbes Travel Guide Five-Star Awards, AAA five diamond, Mobil five-star, Michelin five star, and Zagat Survey Top U.S. Hotel ratings, as well as one Michelin star for its restaurant Wing Lei, the first Chinese restaurant to receive a Michelin Star in the country. \n\nThe Wynn is a genre almost entirely their own. Talk about a blend of opulent creativity and seductive comfort. Wynn Hotel designer Roger Thomas calls it "Evocatect." The "architecture that evokes wonderful moments in people\'s lives."

 > First, I filtered the RDD by the stars column (keeping only reviews that gave over 4 stars, filtering out negative reviews).  I then called ```OrderBy``` to Order the reviews by the 'useful' column and chose ```ascending=False``` so the most useful reviews would be ranked first.  I then called ```take(10)``` to take the top ten rows (most useful positive reviews).

### <font color="magenta">Q4: During what hour of the day do most checkins occur?</font>

In [21]:
checkin.take(1)[0]

Row(business_id='--1UhMGODdWsrMastO9DZw', date='2016-04-26 19:49:16, 2016-08-30 18:36:57, 2016-10-15 02:45:18, 2016-11-18 01:54:50, 2017-04-20 18:39:06, 2017-05-03 17:58:02')

#### Answer:

In [22]:
datesplit = udf(lambda x: x.split(', '),ArrayType(StringType()))

In [23]:
(checkin \
        #apply udf and alias a new column name for the result
        .select('business_id', datesplit('date').alias('dates'))
 
        #create a new column of the exploded dates
        .withColumn('checkin_date', explode('dates'))
 
        #create a new column from the extracted hour from each date
        .withColumn('checkin_hour', hour('checkin_date'))
 
        #group all checkin_hours together
        .groupBy('checkin_hour')    
 
        #count the number of checkins each hour in the df
        .count()                             
 
        #sort the checkin counts by the count (greatest to least)
        .sort('count', ascending=False)  
 
        #show the result :)
        .show()                                                 
)

+------------+-------+
|checkin_hour|  count|
+------------+-------+
|           1|1561788|
|          19|1502271|
|           0|1491176|
|           2|1411255|
|          20|1350195|
|          23|1344117|
|          18|1272108|
|          22|1257437|
|          21|1238808|
|           3|1078939|
|          17|1006102|
|          16| 852076|
|           4| 747453|
|          15| 617830|
|           5| 485129|
|          14| 418340|
|           6| 321764|
|          13| 270145|
|           7| 231417|
|          12| 178910|
+------------+-------+
only showing top 20 rows



> It looks like most check-ins occur at 1:00 AM (I assume this is UTC, which is 9:00 PM EST).  As mentioned in the comments above, first, I applied the user-defined function (```udf```) ```datesplit``` (same as the Spark Demo Notebook) to turn the string of checkin dates into an iterable (a list) of the dates.  I then called the ```alias()``` function to give the result a new column name ```dates```.  I then used ```withColumn``` to create a new column through the function ```explode``` which iterates through the list in the column ```dates``` and for each element of the list creates a new row in the DataFrame.  I then used ```withColumn``` again, combined with the ```hour()``` function to extract the hour from the datetime string in each ```dates``` row.  I then used ```groupBy``` to combine all common hours together in the new column ```checkin_hour```, called the ```count()``` function and then sorted the check_in hours greatest to least based on the checkin frequency by hour.

### <font color="magenta">Q5: Sentiment analysis</font>

a. List the 50 most common non-stopword words that are unique to *positive* reviews.
b. List the 50 most common non-stopword words that are unique to *negative* reviews.

You can use the stopword list that was introduced in the lecture materials or you can 
find/devise one of your own.

You will need to define what constitutes a positive review and what constitutes a negative review.  We highly recommend that you use the number of stars to figure this out.  Be sure to provide a rationale for your choice

As an example, consider the following two reviews:

* Positive: The meal was great, and the service was the best we ever experienced.
* Negative: The meal was awful.  It was the worst thing we ever experienced.

Assume our stopwords are {'the','was','and','the','was','we','it'}

* Positive unique: {'great', 'service', 'best'}

* Negative unique: {'awful', 'worst', 'thing'}

In this example, each unique word occurs just once, so the concept of "top 50" doesn't make sense.  For your data, you'll need to count the number of times each unique word occurs.

### extracting words and removing stopwords

In [24]:
#filter out stopwords
from pyspark.ml.feature import StopWordsRemover

stopword_remover = StopWordsRemover(inputCol="words", outputCol="words_filtered")

In [25]:
#extractswords using the pattern provided
from pyspark.ml.feature import RegexTokenizer

regexTokenizer = RegexTokenizer(inputCol="text", outputCol="words", pattern="[A-Za-z']+", gaps=False)

### positive reviews

In [26]:
reviews_positive = \
   (review
          #filter out reviews under 4 stars
          .filter(review['stars']> 3)
    
           #select two relevant columns from the datafrome
          .select(['stars','text'])
    )

reviews_positive

DataFrame[stars: double, text: string]

In [27]:
#apply the regextokenizer to turn the string of words in the 'text' of the review to a list of words
#a new column is created called "words" as shown in the output
regexTokenized_positive = regexTokenizer.transform(reviews_positive)

regexTokenized_positive

DataFrame[stars: double, text: string, words: array<string>]

In [28]:
#remove stopwords from the list of words determined
pos_words_filtered = stopword_remover.transform(regexTokenized_positive)

pos_words_filtered

DataFrame[stars: double, text: string, words: array<string>, words_filtered: array<string>]

In [29]:
positive_review_words = \
    (pos_words_filtered
                        #select the words_filtered column
                        .select('words_filtered')
     
                        #create a new column with exploded words
                        .withColumn('word', explode('words_filtered'))
     
                        #group by the word column
                        .groupBy('word')
     
                        #count the occurances of each word in the dataframe
                        .count()         
     
                        #sort the words by their frequency in the dataframe
                        .sort('count', ascending=False)               
     )

positive_review_words.show(20)

+--------+-------+
|    word|  count|
+--------+-------+
|   great|2417972|
|   place|2121980|
|    good|2116810|
|    food|1990706|
| service|1426307|
|    time|1364825|
|    like|1290539|
|     one|1212285|
|     get|1198172|
|    back|1115380|
|  really|1072418|
|      go|1071680|
|    also| 992098|
|    best| 951816|
|    love| 894950|
|  always| 856346|
|    nice| 846954|
|friendly| 809624|
|    well| 804011|
| amazing| 781619|
+--------+-------+
only showing top 20 rows



### negative reviews

In [31]:
reviews_negative = (review.filter(review['stars']< 3)    #filter out reviews under 3 stars
                          .select(['stars','text'])      #select two relevant columns from the datafrome
                   )

reviews_negative

DataFrame[stars: double, text: string]

In [32]:
#apply the regextokenizer to turn the string of words in the 'text' of the review to a list of words within it
regexTokenized_negative = regexTokenizer.transform(reviews_negative)

regexTokenized_negative

DataFrame[stars: double, text: string, words: array<string>]

In [33]:
#remove stopwords from the list of words determined
neg_words_filtered = stopword_remover.transform(regexTokenized_negative)

neg_words_filtered

DataFrame[stars: double, text: string, words: array<string>, words_filtered: array<string>]

In [34]:
negative_review_words = \
    (neg_words_filtered
                       #select the words_filtered column
                       .select('words_filtered')                     
                       #create a new column with exploded words
                       .withColumn('word', explode('words_filtered'))
                       #group by the word column
                       .groupBy('word')       
                       #count the occurances of each word in the dataframe
                       .count()             
                       #sort the words by their frequency in the dataframe
                       .sort('count', ascending=False)              
    )

negative_review_words.show(20)

+-------+------+
|   word| count|
+-------+------+
|   food|877721|
|    get|746011|
|  place|734846|
|service|726594|
|    one|717197|
|   like|703995|
|   time|703768|
|   back|689827|
|   good|562820|
|     us|550936|
|     go|533843|
|   even|517995|
|   said|488881|
|  never|474564|
|   told|449981|
|    got|437768|
|  order|399585|
|   came|373626|
|  asked|366369|
|   went|361685|
+-------+------+
only showing top 20 rows



### Finding the Unique Words (running on whole dataframe)

In [31]:
#finding the 50 most common words in positive reviews that are not in any of the negative reviews.

(positive_review_words.join(negative_review_words, on="word", how="leftanti")
                      .sort('count', ascending=False)
                      .show(50)
)

+-------------+-----+
|         word|count|
+-------------+-----+
|        eloff|  291|
|         jabz|  203|
|  ahhhhmazing|  161|
|       fixler|  158|
|          ceg|  139|
|    heartwood|  138|
|       popbar|  138|
|       boothe|  127|
|         emme|  127|
|        artur|  124|
|    delizioso|  119|
|        safak|  116|
| shutterbooth|  109|
|       sidell|  109|
|         homa|  100|
|          imr|   99|
|     perfects|   99|
|       exquis|   98|
|         jayd|   94|
|    ahhmazing|   93|
|         meux|   92|
|      hobgood|   92|
|    merveille|   92|
|        ellkb|   91|
|       sdpnbq|   91|
|      koshari|   91|
|        trego|   86|
|       omalza|   85|
|        hubba|   85|
|scrumptiously|   84|
|      kikuchi|   82|
|   deeeeelish|   81|
|       hiroba|   79|
|        saura|   79|
|      skaught|   78|
|         etti|   78|
|       saskia|   77|
|      poprock|   73|
|      bombbbb|   73|
|        feutr|   73|
|     cranmore|   72|
|       winnah|   72|
|  succule

In [32]:
#finding the 50 most common words in negative reviews that are not in any of the positive reviews

(negative_review_words.join(positive_review_words, on="word", how="leftanti")
                      .sort('count', ascending=False)
                      .show(50)
)

+---------------+-----+
|           word|count|
+---------------+-----+
|   ripoffreport|   72|
|   unsubscribed|   53|
|consumeraffairs|   41|
|   unempathetic|   38|
|       'manager|   37|
|   'supervisor'|   32|
|         pesimo|   31|
|    coinsurance|   31|
|           fahw|   30|
|      discusted|   30|
|         voiers|   30|
|         sorrys|   30|
|     unlawfully|   28|
|         fakest|   28|
|            amj|   27|
|     nonpayment|   27|
|       ontrac's|   26|
|           cfpb|   26|
|         theifs|   26|
|       horibble|   26|
|            hra|   25|
|            ybt|   25|
|     magistrate|   24|
|     transunion|   23|
|    voicemail's|   23|
|           azag|   23|
|        horable|   22|
|       comenity|   21|
|        frauded|   21|
|        suppost|   20|
|  unsubscribing|   20|
|          nedia|   20|
|       cobraron|   20|
|     rescission|   19|
|         rebill|   19|
|      fraudster|   19|
|   incompetance|   19|
|        grossss|   19|
|        searshc

> From the Apache Spark documentation, I gathered a way to use Feature Transformers to use ```RegexTokenizer``` to use a a given regex pattern to to create a new column that is filled with a list of words extracted from the input column's string and ```StopwordsRemover``` to create a new column that removes from the input column accepted stopwords from Spark's database.  To get a dataframe of the most common words to positive and negative reviews, I use a similar process to the last exercise where I called ```explode``` on the column with the list of words used in each review, performed a ```groupBy``` to group all common words and then called ```.count()```.  I then sorted the column by the ```count``` greatest to least.  To get the words unique to positive reviews, I performed a join using "leftanti" to get the words that are in positive reviews but not in negative reviews.